In [4]:
def get_new_feature_name_df(old):
    import pandas as pd
    dup = pd.DataFrame(data=old.groupby('column_name').cumcount(), columns=['dup_cnt'])
    dup = dup.reset_index()
    new = pd.merge(old.reset_index(), dup, how='outer')
    new['column_name'] = new[['column_name', 'dup_cnt']].apply(lambda x: x[0]+'_'+str(x[1]) if x[1]>0 else x[0], axis=1)
    new.drop(columns=['index'], inplace=True)
    return new

def get_human_dataset():
    import pandas as pd
    feature_name_df = pd.read_csv('./human_activity/features.txt', sep='\s+', header=None, names=['column_index','column_name'])
    new_name = get_new_feature_name_df(feature_name_df)
    feature_name = new_name.iloc[:,1].values.tolist()
    X_train = pd.read_csv('./human_activity/train/X_train.txt', sep='\s+', names=feature_name)
    X_test = pd.read_csv('./human_activity/test/X_test.txt', sep='\s+', names=feature_name)
    y_train = pd.read_csv('./human_activity/train/y_train.txt', sep='\s+', names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt', sep='\s+', names=['action'])
    return X_train, X_test, y_train, y_test 

In [3]:
X_train, X_test, y_train, y_test = get_human_dataset()

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [6]:
rf_clf = RandomForestClassifier(random_state=0, max_depth=8) #max_depth를 조절하여 과적합 방지 가능
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy_score(y_test, pred)

C:\Users\user\AppData\Local\Temp\ipykernel_15732\2310348679.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf.fit(X_train, y_train)


0.9195792331184255

In [7]:
from sklearn.model_selection import GridSearchCV

In [10]:
%%time
param = {
    'max_depth':[8,16,24],
    'min_samples_split':[1,6,12],
    'min_samples_leaf':[2,8,16]
}
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=param, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train)
print(grid_cv.best_params_)
print(grid_cv.best_score_)

{'max_depth': 16, 'min_samples_leaf': 8, 'min_samples_split': 6}
0.9178454842219804
CPU times: total: 11.6 s
Wall time: 25.1 s


In [11]:
pred = grid_cv.best_estimator_.predict(X_test)
accuracy_score(y_test, pred)

0.9222938581608415

In [13]:
rf_clf = RandomForestClassifier(n_estimators=500, random_state=0, n_jobs=-1, 
                                max_depth=16, min_samples_leaf=8, min_samples_split=6)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy_score(y_test, pred)

0.9192399049881235

In [14]:
from sklearn.ensemble import GradientBoostingClassifier

In [15]:
%%time
gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
pred = gb_clf.predict(X_test)
accuracy_score(y_test, pred)
#GradientBoosting(GBM)은 시간이 오래 걸린다는 단점이 있음

CPU times: total: 14min 25s
Wall time: 14min 25s


0.9389209365456397